In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.autograd import Variable

import time
import pickle
import numpy as np
import argparse
import datetime
import itertools
import seaborn as sns
from sklearn import metrics
from matplotlib import pyplot as plt

import modules
from utils import dataset
from imp import reload
from modules import encoder_CIFAR10, generator_CIFAR10, discriminator_CIFAR10
from modules import encoder_MNIST, generator_MNIST, discriminator_MNIST

/home/laya/anaconda3/envs/t13/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
device = torch.device('cuda')

In [ ]:
learning_rate = 1e-4
beta1 = 0.5
beta2 = 0.999
num_epochs = 500
latent_size_CIFAR10 = 100
latent_size_MNIST = 100
acc_lam = {}

# CIFAR10

In [ ]:
normalize_CIFAR10 = torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
norm_transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), normalize_CIFAR10])
transform_CIFAR10 = torchvision.transforms.Compose([torchvision.transforms.Resize((35,35)),
                                torchvision.transforms.FiveCrop(32),
                                torchvision.transforms.Lambda(lambda crops: torch.stack([norm_transform(crop)
                                for crop in crops]))])

In [ ]:
train_CIFAR10 = torchvision.datasets.CIFAR10('./data/CIFAR10/', train=True, 
                                            transform=transform_CIFAR10, target_transform=None, download=False)
test_CIFAR10 = torchvision.datasets.CIFAR10('./data/CIFAR10/', train=False, 
                                            transform=transform_CIFAR10, target_transform=None, download=False)

In [ ]:
idx = torch.tensor(train_CIFAR10.targets) == torch.tensor(0)
dset_train = torch.utils.data.dataset.Subset(train_CIFAR10, np.where(idx==True)[0])
dset_train_anomalous = torch.utils.data.dataset.Subset(train_CIFAR10, np.where(idx==False)[0])
trainloader = torch.utils.data.DataLoader(dset_train, batch_size=1, shuffle=True)
print (len(trainloader))

In [ ]:
idx = torch.tensor(test_CIFAR10.targets) == torch.tensor(0)
dset_test = torch.utils.data.dataset.Subset(test_CIFAR10, np.where(idx==True)[0])
dset_test_anomalous = torch.utils.data.dataset.Subset(test_CIFAR10, np.where(idx==False)[0])
testloader_normal = torch.utils.data.DataLoader(dset_test, batch_size=1, shuffle=True)
testloader_anomalous = torch.utils.data.ConcatDataset([dset_train_anomalous, dset_test_anomalous])
testloader_anomalous = torch.utils.data.DataLoader(testloader_anomalous, batch_size=1, shuffle=True)
print (len(testloader_normal), len(testloader_anomalous))

In [ ]:
dis_criterion = nn.CrossEntropyLoss()
aen_criterion = nn.MSELoss()

In [ ]:
enc = encoder_CIFAR10.Encoder(latent_size_CIFAR10)
gen = generator_CIFAR10.Generator(latent_size_CIFAR10)
dis = discriminator_CIFAR10.Discriminator(latent_size_CIFAR10)

In [ ]:
gen.load_state_dict(torch.load('./models/CIFAR10/ALAD_way/8/CIFAR1025epochs2020-03-26-15-11-30G'))
enc.load_state_dict(torch.load('./models/CIFAR10/ALAD_way/8/CIFAR1025epochs2020-03-26-15-11-30E'))
dis.load_state_dict(torch.load('./models/CIFAR10/ALAD_way/8/CIFAR1025epochs2020-03-26-15-11-30D'))

In [ ]:
dis.to(device)
enc.to(device)
gen.to(device);

In [ ]:
lam = 0.2
betha = 1
loss_neg = torch.zeros((len(testloader_normal),1)).cuda()
loss_pos = torch.zeros((len(testloader_anomalous),1)).cuda()

c_neg = c_pos = 0
for step, (images, labels) in enumerate(testloader_normal, 0):
    images = images.view(-1, 3, 32, 32)
    dis.eval()
    gen.eval()
    enc.eval()
    x_real_test = images.cuda()
    E_x = enc(x_real_test)
    G_z = gen(E_x)
    
    E_G_z = enc(G_z)
    
    real, internal_real = dis(x_real_test)
    fake, internal_fake = dis(G_z)
    
    latentloss = torch.mean(torch.abs(E_x - E_G_z))
    resloss = torch.mean(torch.abs(x_real_test - G_z))
    discloss = torch.mean(torch.abs(internal_real-internal_fake))
    loss_test = (1 - lam) * resloss + lam * discloss + betha * latentloss
    loss_neg[c_neg] = loss_test.detach()
    c_neg += 1
    
for step, (images, labels) in enumerate(testloader_anomalous, 0):
    images = images.view(-1, 3, 32, 32)
    dis.eval()
    gen.eval()
    enc.eval()
    x_real_test = images.cuda()
    E_x = enc(x_real_test)
    G_z = gen(E_x)
    
    E_G_z = enc(G_z)
    
    real, internal_real = dis(x_real_test)
    fake, internal_fake = dis(G_z)
    
    latentloss = torch.mean(torch.abs(E_x - E_G_z))
    resloss = torch.mean(torch.abs(x_real_test - G_z))
    discloss = torch.mean(torch.abs(internal_real-internal_fake))
    loss_test = (1 - lam) * resloss + lam * discloss + betha * latentloss
    loss_pos[c_pos] = loss_test.detach()
    c_pos += 1

print ('mean negative: %0.4f, std negative: %0.4f' %(torch.mean(loss_neg), torch.std(loss_neg)))
print ('mean positive: %0.4f, std positive: %0.4f' %(torch.mean(loss_pos), torch.std(loss_pos)))

In [ ]:
x1 = loss_neg.cpu().numpy()
x2 = loss_pos.cpu().numpy()
sns.distplot(x1, hist=False, kde=True, kde_kws={'linewidth': 3}, label='Normal')
sns.distplot(x2, hist=False, kde=True, kde_kws={'linewidth': 3}, label='Anomalous')
plt.title('Distribution of normal and abnormal samples')
plt.xlabel('Anomaly Score');

FP = TP = []
neg_pre_wrong = 0
for i in range(len(loss_neg)):
    if loss_neg[i] > 0.08:
        neg_pre_wrong += 1

pos_pre_wrong = 0
for i in range(len(loss_pos)):
    if loss_pos[i] <= 0.08:
        pos_pre_wrong += 1
print ("number of normal samples missclassified: %d, number of anomalous samples missclassified: %d" 
       %(neg_pre_wrong, pos_pre_wrong))
tp = (len(loss_pos) - pos_pre_wrong)
fn = pos_pre_wrong
fp = neg_pre_wrong
tn = len(loss_neg) - neg_pre_wrong
precision = tp / (tp + fp)
## recall / sensitivity / True Positive Rate
recall = tp / (tp + fn)
## False Positive Rate / 1 - Specificity
fp_rate = fp / (fp + tn)
specificity = tn / (tn + fp)
f1 = 2 * ((precision * recall)/(precision + recall))
accuracy = (tp + tn) / (tp + tn + fp + fn)
acc_lam[lam] = accuracy
print ("tp: %d, fp: %d, fn: %d, tn: %d" %(tp, fp, fn, tn))
print ("precision: %.5f, recall: %.5f, specificity: %.5f, f1: %.5f, fp_rate: %.5f, accuracy: %.5f" 
       %(precision, recall, specificity, f1, fp_rate, accuracy))
anomalous = torch.ones((len(loss_pos), 1))
normal = torch.zeros((len(loss_neg), 1))
y = torch.cat((anomalous, normal), 0)
scores = torch.cat((loss_pos, loss_neg), 0)
fpr, tpr, thresholds = metrics.roc_curve(y.cpu(), scores.cpu())
prec, rec, _ = metrics.precision_recall_curve(y.cpu(), scores.cpu())
average_precision = metrics.average_precision_score(y.cpu(), scores.cpu())
auc = metrics.auc(fpr, tpr)
print ('AUC', auc)
print ('average precision :', average_precision)
plt.figure()
plt.plot(fpr, tpr, color='darkorange', label='ROC curve (area = %0.3f)' % auc)
plt.plot([0.0, 1.0], color='navy', linestyle='--')
plt.xlim([-0.01, 1.0])
plt.ylim([-0.01, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.show();

# MNIST

In [ ]:
normalize_MNIST = torchvision.transforms.Normalize((0.5, ), (0.5, ))
transform_MNIST = torchvision.transforms.Compose([torchvision.transforms.Resize((33, 33)),
                                                torchvision.transforms.RandomVerticalFlip(),
                                                torchvision.transforms.RandomCrop(28),
                                                torchvision.transforms.ToTensor(),
                                                normalize_MNIST])

In [ ]:
train_MNIST = torchvision.datasets.MNIST('./data/MNIST/', train=True,
                                         transform=transform_MNIST, target_transform=None, download=False)
test_MNIST = torchvision.datasets.MNIST('./data/MNIST/', train=False,
                                            transform=transform_MNIST, target_transform=None, download=False)

In [ ]:
idx = torch.as_tensor(train_MNIST.targets) == torch.tensor(0)
dset_train = (torch.utils.data.dataset.Subset(train_MNIST, np.where(idx != 0)[0]))
dset_train_anomalous = torch.utils.data.dataset.Subset(train_MNIST, np.where(idx == 0)[0])

trainloader = torch.utils.data.DataLoader(dset_train, batch_size=1, shuffle=True)
print (len(trainloader))

In [ ]:
idx = torch.as_tensor(test_MNIST.targets) == torch.tensor(0)
dset_test = torch.utils.data.dataset.Subset(test_MNIST, np.where(idx != 0)[0])
dset_test_anomalous = torch.utils.data.dataset.Subset(test_MNIST, np.where(idx == 0)[0])
testloader_normal = torch.utils.data.DataLoader(dset_test, batch_size=1, shuffle=True)
testloader_anomalous = torch.utils.data.ConcatDataset([dset_train_anomalous, dset_test_anomalous])
testloader_anomalous = torch.utils.data.DataLoader(testloader_anomalous, batch_size=1, shuffle=True)
print (len(testloader_normal))
print (len(testloader_anomalous))

In [ ]:
reload(encoder_MNIST)
reload(generator_MNIST)
reload(discriminator_MNIST)
enc = encoder_MNIST.Encoder(latent_size_MNIST)
gen = generator_MNIST.Generator(latent_size_MNIST)
dis = discriminator_MNIST.Discriminator(latent_size_MNIST)

In [ ]:
gen = (torch.load('./models/MNIST/Gaussian/0/MNIST50epochs2020-03-03-21-06-17G'))
enc = (torch.load('./models/MNIST/Gaussian/0/MNIST50epochs2020-03-03-21-06-17E'))
dis = (torch.load('./models/MNIST/Gaussian/0/MNIST50epochs2020-03-03-21-06-17D'))

In [ ]:
gen.load_state_dict(torch.load('./models/CIFAR10/ALAD_way/0/CIFAR1030epochs2020-03-19-11-20-27G'))
enc.load_state_dict(torch.load('./models/CIFAR10/ALAD_way/0/CIFAR1030epochs2020-03-19-11-20-27E'))
dis.load_state_dict(torch.load('./models/CIFAR10/ALAD_way/0/CIFAR1030epochs2020-03-19-11-20-27D'))

In [ ]:
dis.to(device)
enc.to(device)
gen.to(device);

In [ ]:
lam = 0.1
betha = 0.1
loss_neg = torch.zeros((len(testloader_normal),1)).cuda()
loss_pos = torch.zeros((len(testloader_anomalous),1)).cuda()
c_neg = c_pos = 0
start_time = time.time()
for step, (images, labels) in enumerate(testloader_normal, 0):
    images = images.view(-1, 1, 28, 28)
    dis.eval()
    gen.eval()
    enc.eval()
    x_real_test = images.cuda()
    E_x = enc(x_real_test)
    G_z = gen(E_x)
    E_G_z = enc(G_z)
    
    real, internal_real = dis(x_real_test)
    fake, internal_fake = dis(G_z)
    
    latentloss = torch.mean(torch.abs(E_x - E_G_z))
    resloss = torch.mean(torch.abs(x_real_test - G_z))
    discloss = torch.mean(torch.abs(internal_real-internal_fake))
    loss_test = (1 - lam) * resloss + lam * discloss + betha * latentloss
    loss_neg[c_neg] = loss_test.detach()
    c_neg += 1

for step, (images, labels) in enumerate(testloader_anomalous, 0):
    images = images.view(-1, 1, 28, 28)
    dis.eval()
    gen.eval()
    enc.eval()
    x_real_test = images.cuda()
    E_x = enc(x_real_test)
    G_z = gen(E_x)
    E_G_z = enc(G_z)
    
    real, internal_real = dis(x_real_test)
    fake, internal_fake = dis(G_z)
    
    latentloss = torch.mean(torch.abs(E_x - E_G_z))
    resloss = torch.mean(torch.abs(x_real_test - G_z))
    discloss = torch.mean(torch.abs(internal_real-internal_fake))
    loss_test = (1 - lam) * resloss + lam * discloss + betha * latentloss
    loss_pos[c_pos] = loss_test.detach()
    c_pos += 1
end_time = time.time() - start_time
# print ('model time: ', end_time)
print ('mean negative: %0.4f, std negative: %0.4f' %(torch.mean(loss_neg), torch.std(loss_neg)))
print ('mean positive: %0.4f, std positive: %0.4f' %(torch.mean(loss_pos), torch.std(loss_pos)))

In [ ]:
x1 = loss_neg.cpu().numpy()
x2 = loss_pos.cpu().numpy()
sns.distplot(x1, hist=False, kde=True, kde_kws={'linewidth': 3}, label='Normal')
sns.distplot(x2, hist=False, kde=True, kde_kws={'linewidth': 3}, label='Anomalous')
plt.title('Distribution of normal and abnormal samples')
plt.xlabel('Anomaly Score');

FP = TP = []
neg_pre_wrong = 0
for i in range(len(loss_neg)):
    if loss_neg[i] > 0.05:
        neg_pre_wrong += 1

pos_pre_wrong = 0
for i in range(len(loss_pos)):
    if loss_pos[i] <= 0.05:
        pos_pre_wrong += 1
print ("number of normal samples missclassified: %d, number of anomalous samples missclassified: %d" 
       %(neg_pre_wrong, pos_pre_wrong))
tp = (len(loss_pos) - pos_pre_wrong)
fn = pos_pre_wrong
fp = neg_pre_wrong
tn = len(loss_neg) - neg_pre_wrong
precision = tp / (tp + fp)
## recall / sensitivity / True Positive Rate
recall = tp / (tp + fn)
## False Positive Rate / 1 - Specificity
fp_rate = fp / (fp + tn)
specificity = tn / (tn + fp)
f1 = 2 * ((precision * recall)/(precision + recall))
accuracy = (tp + tn) / (tp + tn + fp + fn)
acc_lam[lam] = accuracy
print ("tp: %d, fp: %d, fn: %d, tn: %d" %(tp, fp, fn, tn))
print ("precision: %.5f, recall: %.5f, specificity: %.5f, f1: %.5f, fp_rate: %.5f, accuracy: %.5f" 
       %(precision, recall, specificity, f1, fp_rate, accuracy))
anomalous = torch.ones((len(loss_pos), 1))
normal = torch.zeros((len(loss_neg), 1))
y = torch.cat((anomalous, normal), 0)
scores = torch.cat((loss_pos, loss_neg), 0)
fpr, tpr, thresholds = metrics.roc_curve(y.cpu(), scores.cpu())
auc = metrics.auc(fpr, tpr)
print ('AUC', auc)
plt.figure()
plt.plot(fpr, tpr, color='darkorange', label='ROC curve (area = %0.3f)' % auc)
plt.plot([0.0, 1.0], color='navy', linestyle='--')
plt.xlim([-0.01, 1.0])
plt.ylim([-0.01, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.show();

In [ ]:
for ids, (images, labels) in enumerate(testloader_anomalous):
    print (images[0].shape)
    plt.imshow(images[0].squeeze(0), cmap='gray_r')
    plt.show()
    break

In [ ]:
for ids, (images, labels) in enumerate(testloader_normal):
    print (images[0].shape)
    plt.imshow(images[0].squeeze(0), cmap='gray_r')
    plt.show()
    break